In [1]:
from tifffile import imread
import numpy as np
from stardist.models import StarDist3D
from pathlib import Path

import matplotlib.pyplot as plt

from csbdeep.utils import normalize

from tqdm import tqdm


In [2]:
import sys
sys.path.append('C:/Users/Eric/src/stardist_mpcdf')
from stardist_mpcdf.data import writeDatasetFolders, ImageInterpolation

In [3]:
def allocateOnEmptyGPU():
    import os
    import re
    import numpy as np
    from subprocess import check_output

    nvidia_smi_output = check_output(['nvidia-smi']).decode("utf-8")
    memory_matches = re.findall('\d+MiB\s*/\s*\d+MiB', nvidia_smi_output)
    memory_string = [match.split('MiB')[0] for match in memory_matches]
    gpu_memory_usage = list(map(int, memory_string))

    os.environ['CUDA_VISIBLE_DEVICES'] = str(np.argmin(gpu_memory_usage))
    print('Run on GPU with ID: {}'.format(os.environ['CUDA_VISIBLE_DEVICES']))

    return

In [4]:
allocateOnEmptyGPU()

Run on GPU with ID: 1


In [5]:
X_test_filenames = sorted(Path(r'Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz400\test\images').glob('*.tif'))

In [6]:
Y_test_filenames = sorted(Path(r'Y:\Eric\2021_Iterative_Biofilm_Annotation\datasets\eva-v1-dz400\test\masks').glob('*.tif'))

In [7]:
X_test_filenames

[WindowsPath('Y:/Eric/2021_Iterative_Biofilm_Annotation/datasets/eva-v1-dz400/test/images/im0.tif')]

In [8]:
Y_test_filenames

[WindowsPath('Y:/Eric/2021_Iterative_Biofilm_Annotation/datasets/eva-v1-dz400/test/masks/im0.tif')]

In [9]:
X_test = [imread(x) for x in X_test_filenames]

In [10]:
Y_test = [imread(y) for y in Y_test_filenames]

# Interpolation

In [11]:
dz = 400

In [12]:
if not dz is None:
    factor = int(dz/100)
    X = [ImageInterpolation(x[::factor], factor, x.shape) for x in X_test]

In [13]:
axis_norm = (0, 1, 2)
Y_test= [normalize(x, 1, 99.8, axis=axis_norm) for x in tqdm(Y_test)]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.15it/s]


In [14]:
model = StarDist3D(None, name='eva-v1_dz400_rep1', basedir='models')

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.763609, nms_thresh=0.3.


In [15]:
max_size = 126 # for 11GB GPU
overlap_label = None

In [ ]:
Y_ = []
for x in tqdm(X_test):
    print('Dataset shape: ', x.shape)
    print('Dataset size: ', np.size(x))

    if np.size(x) <= max_size**3: # Limited by 16GB GPU
        y_ = model.predict_instances(x, verbose=True, overlap_label=overlap_label)[0]

    else:
        n_tiles = tuple(np.max([1, s//max_size]) for s in x.shape)
        print('Num tiles: ', n_tiles)
        y_ = model.predict_instances(x, verbose=True, overlap_label=overlap_label,
                                     n_tiles=n_tiles)[0]

    Y_.append(y_)


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Dataset shape:  (176, 543, 526)
Dataset size:  50268768
Num tiles:  (1, 4, 4)



100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:35<00:00,  2.21s/it]


predicting instances with prob_thresh = 0.7636087892716845 and nms_thresh = 0.3
found 484082 candidates


In [ ]:
from tifffile import imsave

In [ ]:
imsave(Y_[0], 'test.tif')